In [ ]:
%pip install google-genai
%pip install os
%pip install dotenv
%pip install pydantic

In [ ]:
from google import genai
from dotenv import load_dotenv
from google.genai import types

In [ ]:
def create_vertexai_client():
    import os
    
    cloud_api_key = os.getenv("GOOGLE_CLOUD_API_KEY")
    if not cloud_api_key:
        raise ValueError("GOOGLE_CLOUD_API_KEY not found in .env file")
    
    # Configure the client with your API key
    client = genai.Client(
        vertexai=True, 
        api_key=cloud_api_key, 
    )

    return client

In [ ]:
load_dotenv()

# Configure the client with your API key
client = create_vertexai_client()

In [ ]:
def get_citations(response: types.GenerateContentResponse) -> list[str]:

    citations: list[str] = []
    if response.candidates is not None and len(response.candidates) > 0:
        candidate = response.candidates[0]
        if candidate.grounding_metadata:
            grounding_supports = candidate.grounding_metadata.grounding_supports or []
            grounding_chunks = candidate.grounding_metadata.grounding_chunks or []
            num_chunks = len(grounding_chunks)
            for support in grounding_supports:
                indices = support.grounding_chunk_indices or []
                for chunk_id in indices:
                    chunk = grounding_chunks[chunk_id] if chunk_id >= 0 and chunk_id < num_chunks else None
                    web_uri = chunk.web.uri if chunk.web is not None else None
                    if web_uri:
                        citations.append(web_uri)

    return citations

In [ ]:
def get_player_stats(player: str) -> types.GenerateContentResponse:
        
    stats = ["goals", "goalAssists", "expectedGoals", "cleanSheets", 
                "yellowCards", "totalRedCards", "blocks", "penaltiesSaved"
                "savesMade", "ownGoalScored", "appearances", "totalTackles",
                "timePlayed"
            ]

    url_list = [f"URL {i}: https://www.premierleague.com/en/stats/top/players?statMetric={stat}&season=2025" 
                for i, stat in enumerate(stats)
            ]
    urls = "\n".join(url_list)

    prompt = f"""
        Please get the season 2025/2026 player stats for this player: {player} from the following URLs.
        {urls}        
    """

    response = client.models.generate_content(
        model='gemini-3-flash-preview',
        contents=types.Content(
            role="user",
            parts=[
                types.Part(text=prompt)
            ]
        ),
        config=types.GenerateContentConfig(
            tools=[
                types.Tool(google_search=types.GoogleSearch()),
                types.Tool(url_context=types.UrlContext()),
            ]
        )
    )
                        
    return response

In [ ]:
response = get_player_stats(player="Leny Yoro")

print(response.text)

In [ ]:
citations = get_citations(response)
for i, citation in enumerate(citations):
     print(f"Citation {i}: {citation}")

In [ ]:
response = get_player_stats(player="Martin Ødegaard")

print(response.text)

In [ ]:
citations = get_citations(response)
for i, citation in enumerate(citations):
     print(f"Citation {i}: {citation}")